## Pourquoi le CAC 40 ?

### Contexte local
- Marché français bien documenté, facilement interprétable dans notre cursus.

### Qualité des données
- Grande capitalisation → meilleure qualité et continuité des prix.

### Intérêt pédagogique
- Volatilité et rotations sectorielles offrent un terrain riche pour tester :
  - **Clustering** (groupes de comportements),
  - **Classification** (profils de risque).

En pratique, les actions du CAC 40 traversent des phases de marché contrastées (incertitudes macro, cycles sectoriels, politique monétaire…), ce qui accentue les différences de profils :
- **Défensifs vs cycliques**,
- **Croissance vs value**.  

👉 C’est idéal pour mettre en évidence les structures “naturelles” via le non supervisé, puis bâtir un modèle supervisé simple et explicable.

---

## Source des prix : Yahoo Finance

Nous utilisons **Yahoo Finance** pour télécharger les prix journaliers des actions.

### Motivations
- Accès libre et pratique (**yfinance**) pour un projet étudiant.  
- Historique suffisamment long et cohérent pour les analyses temporelles.  
- Données ajustées (dividendes/splits) disponibles via *Adj Close*.  

---

## Nettoyage & cohérence temporelle

- Alignement des séries sur les **jours de cotation communs**.  
- Lorsqu’un titre ne cote pas un jour donné (jours fériés, suspension, etc.) :
  - Application d’un **forward-fill (ffill)** avant de calculer les rendements.

### Objectif
- Éviter de créer des trous qui cassent les fenêtres temporelles (volatilité, VaR, etc.).

### Important : pas de biais
- Le ffill **ne regarde que le passé** (aucun recours à des valeurs futures).  
- Pas de *look-ahead bias* → on propage uniquement la dernière information connue à la date *t*.  


In [4]:
# ========= Config =========
import numpy as np
import pandas as pd

TICKERS = [
    "AI.PA",   # Air Liquide
    "AIR.PA",  # Airbus
    "ALO.PA",  # Alstom
    "MT.PA",   # ArcelorMittal
    "ATO.PA",  # Atos
    "CS.PA",   # AXA
    "BNP.PA",  # BNP Paribas
    "CAP.PA",  # Capgemini
    "CA.PA",   # Carrefour
    "ACA.PA",  # Crédit Agricole
    "BN.PA",   # Danone
    "DSY.PA",  # Dassault Systèmes
    "ENGI.PA", # Engie
    "EL.PA",   # EssilorLuxottica
    "RMS.PA",  # Hermès
    "HO.PA",   # Thales (ex Gemalto HO)
    "KER.PA",  # Kering
    "OR.PA",   # L'Oréal
    "MC.PA",   # LVMH
    "ML.PA",   # Michelin
    "ORA.PA",  # Orange
    "PUB.PA",  # Publicis Groupe
    "RI.PA",   # Pernod Ricard
    "RNO.PA",  # Renault
    "SAF.PA",  # Safran
    "SGO.PA",  # Saint-Gobain
    "SAN.PA",  # Sanofi
    "SU.PA",   # Schneider Electric
    "GLE.PA",  # Société Générale
    #"STLA.PA", # Stellantis
    #"STM.PA",  # STMicroelectronics
    "TEP.PA",  # Teleperformance
    "HO.PA",   # Thales
    "TTE.PA",  # TotalEnergies
    #"URW.AS",  # Unibail-Rodamco-Westfield (cotée à Amsterdam mais composant CAC40)
    "VIE.PA",  # Veolia
    "DG.PA",   # Vinci
    "VIV.PA",  # Vivendi
    "WLN.PA",  # Worldline
]

START = "2018-01-01"
END   = "2025-10-07"  

# ========= Download =========
import yfinance as yf

data = yf.download(TICKERS, start=START, end=END, interval="1d", auto_adjust=False, progress=False)

# On ne garde que l'Adj Close (colonnes multi-index -> niveau 0 = 'Adj Close')
if isinstance(data.columns, pd.MultiIndex):
    adj = data["Adj Close"].copy()
else:
    adj = data.copy()

# Harmoniser les jours, supprimer lignes vides totales
adj = adj.dropna(how="all")
# Option : forward-fill sur qq trous isolés
adj = adj.ffill()

# Rendements log (par colonne)
rets = np.log(adj / adj.shift(1)).dropna(how="all")

# Sauvegarde
adj.to_csv("prices_cac40.csv", index=True)
rets.to_csv("returns_cac40.csv", index=True)

print("OK -> prices_cac40.csv et returns_cac40.csv écrits.")

OK -> prices_cac40.csv et returns_cac40.csv écrits.


## Construction des variables (features)

 **Objectif**  
Transformer les séries de prix en **descripteurs financiers** utilisables pour le clustering et la classification.  
###  Rendement moyen annualisé

$$
\mu_{\text{ann}} = \bar{r} \times 252
$$

- \( \bar{r} \) : moyenne des rendements journaliers  
- \( 252 \) : nombre approximatif de jours de bourse dans une année  

---

###  Volatilité annualisée

$$
\sigma_{\text{ann}} = \text{std}(r) \times \sqrt{252}
$$

- \(\text{std}(r)\) : écart-type des rendements journaliers  
- Multiplication par \(\sqrt{252}\) pour annualiser la volatilité  
- Mesure la dispersion des rendements autour de la moyenne  
- Indicateur clé du **risque financier**

---
###  Sharpe annualisé (sans taux sans risque)

$$
S = \frac{\mu_{\text{ann}}}{\sigma_{\text{ann}} + \varepsilon}
$$

- \( \mu_{\text{ann}} \) : rendement moyen annualisé  
- \( \sigma_{\text{ann}} \) : volatilité annualisée  
- \( \varepsilon \) : petite constante pour éviter la division par zéro  

**Raison** : comparer le rendement ajusté du risque sur une base standard (annuelle).  

---

###  VaR 95 % (positive)

On prend le quantile à 5 % des rendements sur la fenêtre (souvent négatif).  
On définit :  

$$
VaR_{95} = - q_{5\%}(r_t)
$$

- Valeur positive = perte attendue au pire 5 % des cas  
- **Convention usuelle** : VaR exprimée en perte  

---

### Drawdown courant  
Perte relative par rapport au dernier plus-haut sur 60 jours :  

$$
DD_t = \frac{P_t}{\max(P_{t-59..t})} - 1 \leq 0
$$

### Max Drawdown (sur 60 jours)  
Minimum des drawdowns dans la fenêtre :  

$$
MDD = \min \left( \frac{P_t}{P_{\text{peak}}} - 1 \right)
$$

**Raison** : mesurer les pertes cumulées (queue de risque) et pas seulement la volatilité.  

---

### 📐 Skewness & Kurtosis

- **Skewness (asymétrie)** : mesure de la dissymétrie de la distribution des rendements  
- **Kurtosis (queue épaisse)** : mesure de l’importance des événements extrêmes  

**Raison** : capturer la forme de la distribution (non-normalité fréquente en finance).  

### 🔗 Bêta (sensibilité au marché) — leave-one-out

$$
\beta_i = \frac{\mathrm{Cov}(r_i, \, r_{m,-i})}{\mathrm{Var}(r_{m,-i}) + \varepsilon}
$$

- \( r_i \) : rendements de l’actif \( i \)  
- \( r_{m,-i} \) : moyenne égal-pondérée des rendements des **autres titres** (excluant \( i \)) sur la fenêtre  
- \( \varepsilon \) : petite constante pour éviter la division par zéro  

**Raison** :  
On retire l’actif \( i \) du portefeuille de marché pour éviter qu’il corrèle mécaniquement avec lui-même → réduit un biais d’estimation.


### Remarques
Choix de conception
- Rolling strict (min_periods=WINDOW) → pas de look-ahead
- Annualisation → comparabilité inter-titres
- VaR positive → convention usuelle
- Deux drawdowns → instantané et maximum
- Bêta leave-one-out → pas de contamination
- Épsilon numérique dans les divisions 
- Pas de remplissage agressif
  
Le bêta incluait le titre lui-même dans le marché, la VaR pouvait être mal interprétée (négative), les drawdowns n’étaient pas toujours cohérents, et certaines statistiques étaient calculées sans annualisation ni garde-fou contre le look-ahead. Nous avons donc corrigé ces points :

Bêta : calcul “leave-one-out” pour éviter qu’un titre s’auto-influence.

VaR95 : exprimée en perte positive, convention claire et standard.

Moyenne, volatilité et Sharpe : annualisés pour comparer équitablement les titres.

Drawdown : deux mesures robustes, l’instantané et le maximum en fenêtre, calculés à partir des prix ajustés.

Rolling strict (min_periods=WINDOW) : garantit que chaque observation n’utilise que le passé (pas de biais).
Ces changements assurent une base de features plus robuste, sans biais d’anticipation, et parfaitement interprétable dans un cadre financier.


In [16]:
# Charger les prix et rendements (déjà préparés en amont, sans ffill agressif)
prices = pd.read_csv("prices_cac40.csv", index_col=0, parse_dates=True)
returns = pd.read_csv("returns_cac40.csv", index_col=0, parse_dates=True)

WINDOW = 60 
ANN_DAYS = 252
EPS = 1e-12  # pour éviter division par 0

# (CHANGEMENT) index de référence = celui des rendements
idx = returns.index

# 1) Marché égal-pondéré (une seule fois) — pas indispensable ici mais on le garde
r_m_all = returns.mean(axis=1)

# Pré-calculs utiles pour "leave-one-out" (sur le même index)
n_assets = returns.count(axis=1)                 # nb d'actifs dispos ce jour-là
sum_assets = returns.sum(axis=1, min_count=1)    # somme des rendements du jour

features = {}

def max_drawdown_np(x: np.ndarray) -> float:
    """Max drawdown sur la fenêtre (valeur négative <= 0)."""
    peak = np.maximum.accumulate(x)
    dd = x / peak - 1.0
    return np.nanmin(dd)

for col in returns.columns:
    r = returns[col]                              # ne pas dropna pour garder l'alignement
    roll = r.rolling(WINDOW, min_periods=WINDOW)

    # 2) Moyenne / vol / Sharpe (annualisés)
    mu = roll.mean()                              # moyenne quotidienne
    sigma = roll.std(ddof=1)                      # std quotidienne
    mu_ann = mu * ANN_DAYS
    sigma_ann = sigma * np.sqrt(ANN_DAYS)
    sharpe_ann = mu_ann / (sigma_ann + EPS)       # Sharpe annualisé

    # 3) VaR 95% (positive, i.e., perte)
    q05 = roll.quantile(0.05)                     # typiquement négatif
    VaR95 = -q05                                  # on stocke une perte > 0

    # 4) Drawdown (courant) et Max Drawdown (fenêtre)
    # (CHANGEMENT) réindexer les PRIX sur l'index des returns pour aligner
    p = prices[col].reindex(idx)
    roll_max_p = p.rolling(WINDOW, min_periods=WINDOW).max()
    dd_current = p / roll_max_p - 1.0

    # max drawdown dans la fenêtre (correct, "in-window")
    mdd_window = p.rolling(WINDOW, min_periods=WINDOW).apply(max_drawdown_np, raw=True)

    # 5) Skew & Kurtosis (roulants)
    skew = roll.skew()
    kurt = roll.kurt()  # Fisher (0 pour normale)

    # 6) Beta leave-one-out: marché excluant l'actif i
    num = sum_assets - r
    den = (n_assets - 1).replace(0, np.nan)       # éviter div/0 le jour où un seul actif cote
    r_m_excl = num / den

    cov = r.rolling(WINDOW, min_periods=WINDOW).cov(r_m_excl)
    var_m = r_m_excl.rolling(WINDOW, min_periods=WINDOW).var()
    beta = cov / (var_m + EPS)

    # forcer l'index commun pour ce bloc de features
    df_feat = pd.DataFrame({
        "mu_ann": mu_ann,
        "sigma_ann": sigma_ann,
        "sharpe_ann": sharpe_ann,
        "VaR95": VaR95,
        "dd_current": dd_current,        # drawdown courant (<=0)
        "mdd_window": mdd_window,        # max drawdown (<=0) sur 60j
        "skew": skew,
        "kurt": kurt,
        "beta_ewm": beta                 # beta vs marché égal-pondéré leave-one-out
    }, index=idx)

    features[col] = df_feat

# Concat MultiIndex (niveau 0 = ticker)
features_all = pd.concat(features, axis=1)

# supprimer les dates où TOUT est NaN (ex: tout début)
features_all = features_all.dropna(how="all")

# démarrer à la première fenêtre complète (évite la 1re ligne quasi vide)
first_full = idx[WINDOW-1]
features_all = features_all.loc[first_full:]

features_all.to_csv("features60_cac40.csv")
print("Features calculées :", features_all.shape)

Features calculées : (1931, 315)


## Standardisation des features

**Pourquoi ?**  
Parce que les variables ne sont pas sur la même échelle :

- $ \mu_{\text{ann}} $  ~ quelques %  
- $ \sigma_{\text{ann}} $  ~ 10–30 %  
- **Kurtosis** ou **Skewness** peuvent être négatifs ou très grands  
- $ VaR_{95} $  en %  
- $ \beta $ souvent ≈ 1 mais variable  

---

 **Sans standardisation** → les méthodes comme **K-Means** ou **Spectral Clustering** seraient dominées par les variables ayant une grande variance.

Dans un **contexte prédictif temporel**, il faut éviter de calculer la moyenne et l’écart-type sur l’ensemble des données (sinon on utilise le futur pour normaliser le passé → look-ahead bias). On doit alors fitter le scaler uniquement sur la partie "train" ou utiliser une normalisation glissante (rolling).  

Dans notre projet, nous sommes dans un premier temps dans un cadre **d’exploration non supervisée (clustering, classification statique)** : on ne cherche pas à prédire hors-échantillon mais à comparer les actions entre elles.  
Donc la standardisation globale de toutes les observations ne pose **aucun biais** et est tout à fait adaptée.  

⚠️ Quand on passera plus tard à une approche supervisée avec split temporel, il faudra travailler du coup avec la standardisation sur la partie "train" uniquement.

In [18]:
from sklearn.preprocessing import StandardScaler

# === Lire correctement le CSV avec MultiIndex de colonnes (ticker, feature) ===
features = pd.read_csv(
    "features60_cac40.csv",
    index_col=0,
    parse_dates=True,
    header=[0, 1]     #clé : deux niveaux d’en-tête
)

# Option : forcer numérique (au cas où)
features = features.apply(pd.to_numeric, errors="coerce")

# === Aplatir les colonnes: ('AI.PA','mu_ann') -> 'AI.PA__mu_ann' ===
features.columns = [f"{c0}__{c1}" for c0, c1 in features.columns]

# -------------------------------
# 1) Scaling global (clustering)
# -------------------------------
scaler_global = StandardScaler()
scaled_global = pd.DataFrame(
    scaler_global.fit_transform(features),
    index=features.index,
    columns=features.columns
)
scaled_global.to_csv("features_scaled_global.csv")
print("features_scaled_global.csv :", scaled_global.shape)

# -------------------------------
# 2) Scaling 75%/25% chronologique (supervisé)
# -------------------------------
n = len(features)
train_size = int(0.75 * n)
train_idx = features.index[:train_size]
test_idx  = features.index[train_size:]

X_train = features.loc[train_idx]
X_test  = features.loc[test_idx]

scaler_split = StandardScaler()
X_train_scaled = scaler_split.fit_transform(X_train)
X_test_scaled  = scaler_split.transform(X_test)

scaled_split = pd.DataFrame(
    np.vstack([X_train_scaled, X_test_scaled]),
    index=features.index,
    columns=features.columns
)
scaled_split.to_csv("features_scaled_split.csv")
print("features_scaled_split.csv :", scaled_split.shape)


✅ features_scaled_global.csv : (1931, 315)
✅ features_scaled_split.csv : (1931, 315)


## EDA
Avant d'aller dans le vif du sujet, il faut faire une exploration statistique des features pour :
- Comprendre ce qu’on manipule  
- Détecter si certaines variables sont inutiles (trop corrélées, bruit, outliers extrêmes)  
- Décider si l’on garde toutes les features ou si certaines doivent être écartées

### 1. Distributions
- Histogrammes et boxplots pour chaque feature  
- **Objectif** : observer la forme des distributions  
  - Gaussienne, asymétrique, queue lourde  
  - Détection d’outliers  

---

### 2. Corrélations
- Calcul de la **matrice de corrélation**  
- Visualisation via une **heatmap** (*seaborn*)  
- Règle pratique : si deux features sont corrélées à > 0.9, on peut en garder une seule pour éviter la redondance  

---

### 3. Outliers
- Scatterplots entre variables clés (ex. volatilité vs Sharpe)  
- Utilisation des **scores-z** pour identifier les valeurs extrêmes  

---

### 4. Décisions possibles
- Supprimer ou conserver certaines features selon leur utilité  
⚠️ Attention : certaines distributions financières sont naturellement asymétriques (**skewness**, **kurtosis**) → cela peut être **informatif** plutôt que du bruit  

In [23]:
# EDA des features (CAC40) - 'features_scaled_global.csv' (colonnes aplaties TICKER__FEATURE)
import os
import matplotlib.pyplot as plt

# --------- Config ---------
CSV_IN = "features_scaled_global.csv"  # (après StandardScaler global)
SAVE_DIR = "eda_figs"
os.makedirs(SAVE_DIR, exist_ok=True)

df = pd.read_csv(CSV_IN, index_col=0, parse_dates=True)

# Convertir les colonnes aplaties -> MultiIndex (ticker, feature)
# ex: 'AI.PA__mu_ann' -> ('AI.PA','mu_ann')
cols = df.columns.astype(str)
tuples = [c.split("__", 1) if "__" in c else ("ALL", c) for c in cols]
df.columns = pd.MultiIndex.from_tuples(tuples, names=["ticker", "feature"])

# --------- 2) Passage au format "long" propre (pas de double stack) ---------
# Résultat: colonnes = ['date','ticker','feature','value']
long_df = (
    df.stack(level=["ticker","feature"], future_stack=True)
      .rename("value")
      .reset_index()
      .rename(columns={"level_0": "date"})
)

# Liste des features disponibles
features_list = sorted(df.columns.get_level_values("feature").unique())

# --------- 3) Distributions (histos + boxplots) par feature ---------
for feat in features_list:
    sub = long_df.loc[long_df["feature"] == feat, "value"].dropna()

    # Histogramme
    plt.figure(figsize=(6,4))
    plt.hist(sub, bins=40, alpha=0.8, edgecolor="k")
    plt.title(f"Distribution - {feat}")
    plt.xlabel("valeur (standardisée)")  # car on lit le fichier 'scaled'
    plt.ylabel("fréquence")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"hist_{feat}.png"), dpi=150)
    plt.close()

    # Boxplot
    plt.figure(figsize=(4,5))
    plt.boxplot(sub, vert=True, showfliers=True)
    plt.title(f"Boxplot - {feat}")
    plt.ylabel("valeur (standardisée)")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"box_{feat}.png"), dpi=150)
    plt.close()

# --------- 4) Matrice de corrélation entre features ---------
# Construire une table (date,ticker) x features
pivot = (
    long_df
    .pivot_table(index=["date","ticker"], columns="feature", values="value", aggfunc="mean")
    .dropna(how="any")  # garder uniquement les lignes complètes pour corrélation
)

corr = pivot.corr()

plt.figure(figsize=(8,6))
im = plt.imshow(corr.values, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(ticks=np.arange(len(corr.columns)), labels=corr.columns, rotation=90)
plt.yticks(ticks=np.arange(len(corr.index)), labels=corr.index)
plt.title("Corrélation entre features (sur observations (date,ticker))")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "corr_features.png"), dpi=150)
plt.close()

# --------- 5) Scatterplots utiles + Outliers (z-score) ---------
def zscore(x):
    return (x - np.nanmean(x)) / (np.nanstd(x) + 1e-12)

# Paires pertinentes (si présentes)
pairs = [
    ("sigma_ann", "sharpe_ann"),
    ("sigma_ann", "VaR95"),
    ("beta_ewm",  "sharpe_ann"),
]

for x_feat, y_feat in pairs:
    if x_feat in pivot.columns and y_feat in pivot.columns:
        X = pivot[[x_feat, y_feat]].dropna()
        zx = zscore(X[x_feat].values)
        zy = zscore(X[y_feat].values)
        z_radius = np.sqrt(zx**2 + zy**2)

        # Scatter
        plt.figure(figsize=(6,5))
        plt.scatter(X[x_feat], X[y_feat], s=10, alpha=0.7)
        plt.xlabel(x_feat); plt.ylabel(y_feat)
        plt.title(f"Scatter: {x_feat} vs {y_feat} (points = (date,ticker))")
        plt.tight_layout()
        plt.savefig(os.path.join(SAVE_DIR, f"scatter_{x_feat}_vs_{y_feat}.png"), dpi=150)
        plt.close()

        # Outliers (par z-score combiné)
        X_out = X.copy()
        X_out["z_radius"] = z_radius
        X_out_sorted = X_out.sort_values("z_radius", ascending=False).head(20)
        X_out_sorted.to_csv(os.path.join(SAVE_DIR, f"outliers_{x_feat}_vs_{y_feat}.csv"))

KeyError: 'date'

In [ ]:
# --------- 6) Résumé rapide en console (propre & informatif) ---------
print("=== EDA terminé ===")
print(f"- Fichier analysé        : {CSV_IN}")
print(f"- Nb d'observations      : {pivot.shape[0]} (couples (date, ticker) après dropna)")
print(f"- Nb de features         : {pivot.shape[1]}")
print(f"- Features               : {', '.join(map(str, pivot.columns.tolist()))}\n")

print(f"- Figures enregistrées dans : {SAVE_DIR}")
print(f"  • Heatmap corrélations : {os.path.join(SAVE_DIR, 'corr_features.png')}")
for feat in features_list:
    print(f"  • Hist/Box {feat}      : hist_{feat}.png / box_{feat}.png")
for x_feat, y_feat in pairs:
    if x_feat in pivot.columns and y_feat in pivot.columns:
        print(f"  • Scatter {x_feat} vs {y_feat} : "
              f"scatter_{x_feat}_vs_{y_feat}.png "
              f"+ outliers_{x_feat}_vs_{y_feat}.csv")

# Paires très corrélées (utile pour décider de supprimer des features redondantes)
thr = 0.90
high = []
for i, a in enumerate(pivot.columns):
    for j, b in enumerate(pivot.columns):
        if j <= i:
            continue
        val = corr.loc[a, b]
        if abs(val) >= thr:
            high.append((a, b, val))

if high:
    print(f"\n- Paires très corrélées (|ρ| ≥ {thr}) :")
    for a, b, v in sorted(high, key=lambda t: -abs(t[2]))[:10]:
        print(f"   • {a} ~ {b} : ρ = {v:.3f}")
else:
    print(f"\n- Aucune paire très corrélée (|ρ| ≥ {thr}).")
